In [1]:
!pip install tensorflow-gpu

     |████████████████████████████████| 320.4MB 53kB/s 


In [3]:
!nvidia-smi

Sun Oct 11 05:47:13 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
 train='/content/drive/My Drive/cotton_disease/train' 
 test='/content/drive/My Drive/cotton_disease/test'
 val='/content/drive/My Drive/cotton_disease/val'

In [5]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [6]:
IMAGE_SIZE = [224, 224]

In [7]:
  # Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

94773248/94765736 [==============================] - 1s 0us/step


In [8]:
# don't train existing weights
for layer in resnet.layers:
    layer.trainable = False

In [9]:
# useful for getting number of output classes
folders = glob('/content/drive/My Drive/cotton_disease/train/*')

In [10]:

# our layers - you can add more if you want
x = Flatten()(resnet.output)

In [11]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=resnet.input, outputs=prediction)

In [12]:
# view the structure of the model
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [13]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [14]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [15]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/My Drive/cotton_disease/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1951 images belonging to 4 classes.


In [20]:
test_set = test_datagen.flow_from_directory('/content/drive/My Drive/cotton_disease/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 18 images belonging to 4 classes.


In [21]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
61/61 [==============================] - 968s 16s/step - loss: 2.2327 - accuracy: 0.4147 - val_loss: 1.5519 - val_accuracy: 0.6111
Epoch 2/20
61/61 [==============================] - 36s 588ms/step - loss: 1.0166 - accuracy: 0.6002 - val_loss: 1.0820 - val_accuracy: 0.6111
Epoch 3/20
61/61 [==============================] - 36s 585ms/step - loss: 0.9753 - accuracy: 0.6135 - val_loss: 0.9468 - val_accuracy: 0.6667
Epoch 4/20
61/61 [==============================] - 36s 584ms/step - loss: 0.8127 - accuracy: 0.6643 - val_loss: 0.8567 - val_accuracy: 0.7222
Epoch 5/20
61/61 [==============================] - 35s 582ms/step - loss: 0.7593 - accuracy: 0.6925 - val_loss: 0.7900 - val_accuracy: 0.7222
Epoch 6/20
61/61 [==============================] - 36s 583ms/step - loss: 0.7674 - accuracy: 0.6930 - val_loss: 1.0196 - val_accuracy: 0.5000
Epoch 7/20
61/61 [==============================] - 36s 592ms/step 